In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt





main_df = pd.read_csv("HSQ/data.csv") #will remain unedited for duration
df = pd.read_csv("HSQ/data.csv") #column to be processed

question_indexes =['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21',
       'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28', 'Q29', 'Q30', 'Q31',
       'Q32']

characteristic_indexes = ['affiliative', 'selfenhancing', 'agressive', 'selfdefeating']



In [16]:
#Before we start - Remove all entries where accuracy is 0 since user does not wish to be included in research

df = df[df.accuracy != 0]
len(df) - len(main_df)
#There are no 0 accuracy values therefore no entries need to be removed

0

In [5]:
df.describe()
# we can see some errors including some ridiculous age ranges and -1 values for age columns

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q30,Q31,Q32,affiliative,selfenhancing,agressive,selfdefeating,age,gender,accuracy
count,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,...,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000
mean,2.025210,3.342670,3.078431,2.833800,3.599440,4.152194,3.277311,2.535014,2.582633,2.869281,...,3.945845,2.767507,2.838469,4.010644,3.375537,2.956583,2.762745,70.966387,1.455649,87.542484
std,1.075782,1.112898,1.167877,1.160252,1.061281,0.979315,1.099974,1.231380,1.224530,1.205013,...,1.135189,1.309601,1.233889,0.708479,0.661533,0.410870,0.645982,1371.989249,0.522076,12.038483
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,1.300000,0.000000,0.000000,0.000000,14.000000,0.000000,2.000000
25%,1.000000,3.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,...,3.000000,2.000000,2.000000,3.600000,2.900000,2.800000,2.300000,18.500000,1.000000,80.000000
50%,2.000000,3.000000,3.000000,3.000000,4.000000,4.000000,3.000000,2.000000,2.000000,3.000000,...,4.000000,3.000000,3.000000,4.100000,3.400000,3.000000,2.800000,23.000000,1.000000,90.000000
75%,3.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,3.000000,3.000000,4.000000,...,5.000000,4.000000,4.000000,4.500000,3.800000,3.300000,3.100000,31.000000,2.000000,95.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.100000,5.000000,5.000000,5.000000,44849.000000,3.000000,100.000000


In [6]:
#checking for any null values
set(df.isnull().sum())
#{0} is output therefore no null values present in data set

{0}

In [7]:
from decimal import localcontext, Decimal, ROUND_HALF_UP

def custom_round(i):

    with localcontext() as ctx:
        ctx.rounding = ROUND_HALF_UP
        n = Decimal(i) * 10 
        return float(n.to_integral_value()/10)

#Questions -1 entries 
#-1 entries must not be deemed a more negative result than 0, value must be replaced
#replacing -1 entries with 3 (average result no preference, middle choice)



#this hence means we have to also change the characteristic columns as well since the values have now changed
#note the formulas have been given in codebook.txt and that php rounds up from 0.05
#custom rounder function is required since python rounding does not work like this


medians = df[question_indexes].median()
# medians = dict.fromkeys(question_indexes, 3)

for value in question_indexes:
    df[value] = np.where(df[value] == -1, medians[value], df[value])


df["affiliative"] = (6-df["Q1"] + df["Q5"] + 6-df["Q9"] + df["Q13"] + 6-df["Q17"] + df["Q21"]+6-df["Q25"] + 6-df["Q29"])/8
df["selfenhancing"] = (df["Q2"] + df["Q6"] + df["Q10"] + df["Q14"] + df["Q18"] + df["Q22"]+df["Q26"] + df["Q30"])/8
df["agressive"] = (df["Q3"] + df["Q7"] + df["Q11"] + df["Q15"] + df["Q19"] + df["Q23"]+df["Q27"] + df["Q31"])/8
df["selfdefeating"] = (df["Q4"] + df["Q8"] + df["Q12"] + df["Q16"] + df["Q20"] + df["Q24"]+df["Q28"] + df["Q32"])/8

for metric in ['affiliative', 'selfenhancing', 'agressive', 'selfdefeating']:
    df[metric] = list((map(custom_round,df[metric])))


In [8]:
#fixing age outliers


age_outliers = []

for value in df.age.unique():
    if value > 120: 
        age_outliers.append(value)
#These are clearly outliers since the oldest human know lived to 120; this is clearly an anomoly
#currently replacing with median of age column 
age_median = df.age.median()
df.age = np.where(df.age > 120, age_median, df.age)

df.age.describe()



count    1071.000000
mean       26.316527
std        11.040863
min        14.000000
25%        18.500000
50%        23.000000
75%        30.500000
max        70.000000
Name: age, dtype: float64

In [9]:

# for value in question_indexes:
#     df[value] = np.where(df[value] == -1, medians[value], df[value])



gender_mode = df.gender.mode() #gender can only be 1,2,3 not 0
df.gender = np.where(df.gender == 0, gender_mode, df.gender)


In [10]:
df.to_csv("HSQ/cleaned_data.csv", index = False)

dx = pd.read_csv("HSQ/cleaned_data.csv")
dx.describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q30,Q31,Q32,affiliative,selfenhancing,agressive,selfdefeating,age,gender,accuracy
count,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,...,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000
mean,2.033613,3.353875,3.085901,2.837535,3.608777,4.161531,3.284781,2.554622,2.593838,2.880486,...,3.983193,2.793651,2.849673,4.010084,3.392250,2.969281,2.772829,26.316527,1.460317,87.542484
std,1.063757,1.088977,1.154470,1.154322,1.042587,0.953623,1.084359,1.198346,1.205253,1.187435,...,1.050900,1.273525,1.217012,0.707999,0.641745,0.387466,0.633548,11.040863,0.513430,12.038483
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.300000,1.000000,1.000000,1.000000,14.000000,1.000000,2.000000
25%,1.000000,3.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,...,3.000000,2.000000,2.000000,3.600000,3.000000,2.800000,2.400000,18.500000,1.000000,80.000000
50%,2.000000,3.000000,3.000000,3.000000,4.000000,4.000000,3.000000,2.000000,2.000000,3.000000,...,4.000000,3.000000,3.000000,4.100000,3.500000,3.000000,2.800000,23.000000,1.000000,90.000000
75%,3.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,3.000000,3.000000,4.000000,...,5.000000,4.000000,4.000000,4.500000,3.800000,3.300000,3.100000,30.500000,2.000000,95.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,70.000000,3.000000,100.000000
